# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
print (geoapify_key)

783121592e364cd0a39c4ecd474714cc


In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/kenuzb/week6/python-api-challenge/WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
#%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 70 degrees but higher than 10
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 70) & (city_data_df["Max Temp"] > 10)]

#Wind speed less than 7 mph
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 7)]

#Cloudiness less than 50%
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 50)]



# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

/var/folders/k3/0l0nvb7131s4m88c4gv8x6280000gn/T/ipykernel_53092/3797166453.py:8: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
/var/folders/k3/0l0nvb7131s4m88c4gv8x6280000gn/T/ipykernel_53092/3797166453.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
2,new norfolk,AU,-42.7826,147.0587,58,
6,albany,US,42.6001,-73.9662,74,
9,saint-pierre,RE,-21.3393,55.4781,68,
19,puerto ayora,EC,-0.7393,-90.3518,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 100
params = {
     "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    
# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
grimmen - nearest hotel: No hotel found
stornoway - nearest hotel: County Hotel
margate - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
mogadishu - nearest hotel: No hotel found
hobart - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
lasa - nearest hotel: No hotel found
vaini - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
busselton - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
bredasdorp - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
ab

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,No hotel found
6,albany,US,42.6001,-73.9662,74,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
19,puerto ayora,EC,-0.7393,-90.3518,82,No hotel found
...,...,...,...,...,...,...
565,wyndham,US,37.6985,-77.6125,52,No hotel found
568,road town,VG,18.4167,-64.6167,70,No hotel found
571,dargaville,NZ,-35.9333,173.8833,83,No hotel found
575,iquique,CL,-20.2208,-70.1431,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)